In [1]:
import numpy as np
import torch
import torch.nn as nn 
from torch.autograd import Variable
import pennylane as qml
import math 
from sympy import *
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from dataset import *

In [2]:
#assuming r=1,
def cartesian_to_spherical(x, y, z):
    theta = math.acos(z / 1)  
    phi = math.atan2(y, x)
    if phi < 0:
        phi += 2*np.pi
    return [theta, phi]


    


def fibonacci_sphere(n):

    points = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians

    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append((x, y, z))
        sph_points = [cartesian_to_spherical(x,y,z) for x,y,z in points]
    
    return sph_points



In [3]:
dtype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class VariationalQuantumClassifierInterface:
    def __init__(
            self,
            num_of_input,
            num_of_output,
            num_of_wires,
            num_of_layers,
            var_Q_circuit,
            var_Q_bias,
            qdevice):

        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias
        self.num_of_input = num_of_input
        self.num_of_output = num_of_output
        self.num_of_wires = num_of_wires
        self.num_of_layers = num_of_layers

        self.qdevice = qdevice

        self.dev = qml.device(self.qdevice, wires = num_of_wires)


    def set_params(self, var_Q_circuit, var_Q_bias):
        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias

    def init_params(self):
        self.var_Q_circuit = Variable(torch.tensor(0.01 * np.random.randn(self.num_of_layers, self.num_of_wires, 3), device=device).type(dtype), requires_grad=True)
        return self.var_Q_circuit

    def _statepreparation(self, angles):

        """Encoding block of circuit given angles

        Args:
            a: feature vector of rad and rad_square => np.array([rad_X_0, rad_X_1, rad_square_X_0, rad_square_X_1])
        """
        # num_of_input determines the number of rotation needed.

        for i in range(self.num_of_input):
            qml.RY(angles[i,0], wires=i)
            qml.RZ(angles[i,1], wires=i)

    def _layer(self, W):
        """ Single layer of the variational classifier.

        Args:
            W (array[float]): 2-d array of variables for one layer

        """

        # Entanglement Layer

        for i in range(self.num_of_wires):
            qml.CNOT(wires=[i, (i + 1) % self.num_of_wires])

        # Rotation Layer
        for j in range(self.num_of_wires):
            qml.Rot(W[j, 0], W[j, 1], W[j, 2], wires=j)

    def circuit(self, angles):

        @qml.qnode(self.dev, interface='torch')
        def _circuit(var_Q_circuit, angles):
            """The circuit of the variational classifier."""
            self._statepreparation(angles)
            weights = var_Q_circuit

            for W in weights:
                self._layer(W)


            k = self.num_of_input-1
            return [qml.expval(qml.PauliZ(k))]

        return _circuit(self.var_Q_circuit, angles)



    def forward(self, angles):
        result = ((self.circuit(angles).item())-1.0)*35.0
        return torch.tensor(result, requires_grad=True)

In [4]:
vqc = VariationalQuantumClassifierInterface(
            num_of_input =12,
            num_of_output =1,
            num_of_wires=12,
            num_of_layers=2,
            var_Q_circuit=None,
            var_Q_bias = None,
            qdevice = "default.qubit")
           
fib_angles = fibonacci_sphere(18)
         

class VQCTorch(nn.Module):
    def __init__(self):
        super().__init__()

        self.q_params = nn.Parameter(0.01 * torch.randn(2, 12, 3))
    def get_angles(self, in_x):
        in_x_int = [int(item) for item in in_x.tolist()]
        angles = []
        for item in in_x_int:
            theta = fib_angles[item][0]
            phi = fib_angles[item][1]
            angles.append([theta, phi])

        return torch.tensor(angles, requires_grad=True)


    def forward(self, batch_item):
        vqc.var_Q_circuit = self.q_params
        output_batch = []

        for single_item in batch_item:
            angles = self.get_angles(single_item)

            q_out_elem = vqc.forward(angles)
            
            output_batch.append(q_out_elem)

        outputs = torch.stack(output_batch).view(len(batch_item), 1) 
        return outputs


In [5]:
params = {'batch_size' : 4, 'lr': 0.01, 'epochs': 100}

In [6]:
 
dataset = PeptideDataset()
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(dataset = train_dataset, batch_size = params['batch_size'], shuffle = True, num_workers = 2)
train_iter = iter(train_loader)
train_data = next(train_iter)
x_train, y_train = train_data

test_loader = DataLoader(dataset = train_dataset, batch_size = params['batch_size'], shuffle = True, num_workers = 2)
test_iter = iter(test_loader)
test_data = next(test_iter)
x_test, y_test = test_data


                          

In [ ]:
model = VQCTorch()

lr = params['lr']

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  

batch_size = params['batch_size']
num_epochs = params['epochs']
total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples/batch_size)
for epoch in range(num_epochs):
    print(f"EPOCH: {epoch}")
    for i, (data, target) in enumerate(train_loader):
        y_predicted = model(data)
        loss = criterion(y_predicted, target)
    
        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        #
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {data.shape} | Labels {target.shape}|loss = {loss.item():.4f}')


        
        



EPOCH: 0
Epoch: 1/100, Step 5/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 71.7826
Epoch: 1/100, Step 10/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 206.1361
Epoch: 1/100, Step 15/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 66.6691
Epoch: 1/100, Step 20/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 246.2763
Epoch: 1/100, Step 25/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 328.6136
Epoch: 1/100, Step 30/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 152.5495
Epoch: 1/100, Step 35/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 167.0822
Epoch: 1/100, Step 40/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 528.9880
Epoch: 1/100, Step 45/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 420.9786
Epoch: 1/100, Step 50/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 163

Epoch: 1/100, Step 415/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 377.7589
Epoch: 1/100, Step 420/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 277.0833
Epoch: 1/100, Step 425/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 219.9514
Epoch: 1/100, Step 430/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 99.1011
Epoch: 1/100, Step 435/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 220.0008
Epoch: 1/100, Step 440/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 501.9571
Epoch: 1/100, Step 445/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 486.2918
Epoch: 1/100, Step 450/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 447.4493
Epoch: 1/100, Step 455/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 444.8477
Epoch: 1/100, Step 460/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 

Epoch: 1/100, Step 825/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 465.2883
Epoch: 1/100, Step 830/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 332.8931
Epoch: 1/100, Step 835/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 291.3441
Epoch: 1/100, Step 840/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 264.1712
Epoch: 1/100, Step 845/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 423.1006
Epoch: 1/100, Step 850/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 302.9996
Epoch: 1/100, Step 855/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 475.5890
Epoch: 1/100, Step 860/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 374.9365
Epoch: 1/100, Step 865/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 176.3221
Epoch: 1/100, Step 870/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss =

Epoch: 1/100, Step 1230/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 233.0379
Epoch: 1/100, Step 1235/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 329.9302
Epoch: 1/100, Step 1240/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 335.2415
Epoch: 1/100, Step 1245/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 383.5779
Epoch: 1/100, Step 1250/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 53.9456
Epoch: 1/100, Step 1255/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 831.7363
Epoch: 1/100, Step 1260/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 182.6016
Epoch: 1/100, Step 1265/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 437.8848
Epoch: 1/100, Step 1270/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 302.9474
Epoch: 1/100, Step 1275/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1

Epoch: 1/100, Step 1635/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 406.6762
Epoch: 1/100, Step 1640/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 327.0101
Epoch: 1/100, Step 1645/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 83.5667
Epoch: 1/100, Step 1650/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 243.4202
Epoch: 1/100, Step 1655/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 97.2367
Epoch: 1/100, Step 1660/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 253.6250
Epoch: 1/100, Step 1665/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 155.0772
Epoch: 1/100, Step 1670/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 333.4644
Epoch: 1/100, Step 1675/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])|loss = 356.8784
Epoch: 1/100, Step 1680/88396| Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1]